In [1]:
import pandas as pd
import numpy as np

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
from lifelines import CoxPHFitter

In [19]:
df1 = pd.read_csv('entrenar_st_1820.csv')
#df2 = pd.read_csv('entrenar_st_1921.csv')
#df3 = pd.read_csv('entrenar_st_2022.csv')
#df4 = pd.read_csv('entrenar_st_2123.csv')
archivos = [df1,df2,df3,df4]

In [23]:
df1.head()

,ref_hash,s_source_id,s_ref_type1,s_t_min,s_t_prom,s_t_max,s_ref_type_7,s_#subastas,e_kind,e_app_id,e_event_id,e_#wifi,e_#atribuidos,e_#eventos,e_t_prom,segundos
0,41863526108385,0.122122,1.0,157228.0,185636.171429,189019.0,0.0,35.0,0.126640,0.211054,0.126202,0.0,0.0,88.0,140062.647727,259200.0
1,135153013040192,0.508547,1.0,187854.0,188381.875000,188652.0,0.0,8.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,259200.000000,259200.0
2,161514654074162,0.249773,1.0,10366.0,10425.333333,10489.0,0.0,6.0,0.057634,0.087007,0.058797,8.0,0.0,8.0,147937.250000,259200.0
3,181891380775191,0.508547,1.0,256765.0,256765.000000,256765.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,259200.000000,259200.0
4,186034136943920,0.508547,1.0,60166.0,160935.142857,257677.0,0.0,7.0,0.148482,0.253877,0.147929,0.0,0.0,13.0,60125.769231,88.0


In [34]:
def buscar_parametros_cox_st(alfa):
    cph = CoxPHFitter(alpha=0.05)
    suma_errores = 0
    for df in archivos:
        X,y = df.drop(['ref_hash'],axis=1),df['segundos']
        X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.2, random_state=150)

        cph.fit(X_train,duration_col='segundos', weights_col='s_t_prom', robust=True)
        preds = cph.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        suma_errores += rmse
       
    error_promedio = suma_errores / len(archivos)
    print('RMSE promedio: %f' % (error_promedio))
    return (error_promedio)


In [35]:
alfa = 0.05
for v in [0.02,0.05,0.07]:
    alfa = v
    print('Alpha: %f'% (v))
    error = buscar_parametros_cox_st(alfa)
    errores.append([error,v])
error,v = min(errores,key=lambda x: x[0])
print('Error minimo %f con valor %f' %(error,v))

Alpha: 0.020000


ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high colinearity. Please see the following tips in the lifelines documentation:
https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
Matrix is singular.

In [ ]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis()
estimator.fit(X,y)